# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [2]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
from io import open
import glob
import numpy as np
import pandas as pd
import os
import unicodedata
import string
import torch
import torch.utils.data
from torch.utils.data import Dataset
import torch.nn as nn
from dga.tools import matsnu
from sklearn.preprocessing import LabelEncoder
# from __future__ import unicode_literals, print_function, division
# global constants
# MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# !ls -la "/srv/app/model/data/"

In [3]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("PyTorch: " + torch.__version__)
if torch.cuda.is_available():
    print(f"There are {torch.cuda.device_count()} CUDA devices available")
    for i in range(0,torch.cuda.device_count()):
        print(f"Device {i:0}: {torch.cuda.get_device_name(i)} ")
else:
    print("No GPU found")

numpy version: 1.18.1
pandas version: 1.0.5
PyTorch: 1.4.0
No GPU found


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [5]:
# should be the name of directory you created to save your features data
data_dir = '../data'

In [6]:
class DomainDataset(Dataset):
    def __init__(self, df, train=True):
        """
        Args:
            data_dir (string): directory name
            csv_filename (string): csv filename
        """
        
        self.data_df = df
        
        self.all_chars =  self.__build__chars__()
        self.inputs = self.data_df.iloc[:, 0]
                                   
        self.train = train
                                   
        if self.train:
            self.labels = self.data_df.iloc[:, -1]
        
        self.data_len = len(self.data_df.index)

    def __build__chars__(self):
        """Build dictionary of chars."""
        all_letters = string.ascii_letters + string.digits + " .'-"
        return {all_letters[i]:i+1 for i in range(0, len(all_letters))}
    
    def char_to_ix(self, char):
        """Character to index lookup."""
        return self.all_chars[char]

    def ix_to_char(self, char):
        """Index to character lookup."""
        for i, val in self.all_chars.items():
            if val == char:
                return i

    def domain_to_ix(self, domain):
        """Domain to sequence of indexes."""
        return torch.LongTensor([self.char_to_ix(i) for i in domain])

    def __getitem__(self, index):
        domain = self.domain_to_ix(self.inputs[index])
        if self.train:
            target = torch.Tensor([self.labels[index]])
            return domain, target
        return domain

    def __len__(self):
        return self.data_len

In [7]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

    return xx_pad, yy, x_lens, y_lens

def _get_train_test_data_loader(batch_size, df):
    print("Getting test and train data loaders.")
    
    dataset =  DomainDataset(df, train=True)
    
    train_size = int(0.8 * len(dataset))
    
    test_size = len(dataset) - train_size
    
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
    
    test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)

    return train_dl, test_dl

def pad_collate_pred(batch):

    x_lens = [len(x) for x in batch]

    xx_pad = pad_sequence(batch, batch_first=True, padding_value=0)

    return xx_pad, x_lens

def _get_predict_loader(batch_size, df):
    print("Getting test and train data loaders.")
    
    dataset =  DomainDataset(df, train=False)
    
    predict_dl = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_pred)
    
    return predict_dl

In [8]:
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

class DGAClassifier(nn.Module):
    """
    RNN Estimator for generating sequences of target variables.
    """
    
    def __init__(self, input_features=65, hidden_dim=12, n_layers=2, output_dim=1, embedding_dim=5, batch_size=10):
        super(DGAClassifier, self).__init__()

        # Variables
        self.hidden_dim = hidden_dim
        self.hidden_layers = n_layers
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        
        # Embedding
        self.embedding = nn.Embedding(input_features, embedding_dim)
        
        # RNN Layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=0.3, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, x_lens):
        """
        Perform a forward pass of our model on batch of tracks.
        """
        
        # x: (batch_size, longest_sequence, embedding) i.e. 10, 32, 5
        # hidden size: (hidden_layers, batch_size, hidden_dim) i.e. 2, 10, 30
        batch_size, seq_len = x.size()

        # x_embed: (batch_size, longest_sequence, 1?, embedding_size)
        embed_x = self.embedding(x)
        
        x_packed = pack_padded_sequence(embed_x, x_lens, batch_first=True, enforce_sorted=False)

        # Passing in the input and hidden state into the model and obtaining outputs
        output_packed, hidden_state = self.rnn(x_packed)
        
        output_padded, lengths = pad_packed_sequence(output_packed, batch_first=False)
        
        output = output_padded.view(batch_size*seq_len, self.hidden_dim)
        
        adjusted_lengths = [(l-1)*batch_size + i for i, l in enumerate(lengths)]
        
        lengthTensor = torch.tensor(adjusted_lengths, dtype=torch.int64)
        
        output = output.index_select(0, lengthTensor)
        
        output = output.view(batch_size, self.hidden_dim)
        
        output = self.sigmoid(self.fc(output))
        
        return output

In [9]:
def seed_everything(seed, cuda=False):
    # Set the random seed manually for reproducibility.
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

In [15]:
# TO:DO Figure out splunk query
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [16]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("DGA_App")
print(param)
print(df.describe)

{'options': {'params': {'mode': 'stage', 'algo': 'rnn_dga_classifier', 'epochs': '2', 'batch_size': '32'}, 'args': ['domain', 'label'], 'feature_variables': ['domain', 'label'], 'model_name': 'DGA_App', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '1000', 'max_model_size_mb': '15', 'max_score_time': '600', 'streaming_apply': 'false', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['domain', 'label']}
<bound method NDFrame.describe of                                   domain  label
0       dustindicatefaultpressgarden.com      1
1          parkschedulebuildpencatch.com      1
2         newsattemptexperiencedtake.com      1
3           wingconditionpasslecture.com      1
4      fooddishwitnessboxtankoperate.com      1
...                      

In [ ]:
df

In [12]:
# # # THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
# train_dl, test_dl = _get_train_test_data_loader(batch_size, df) # print(df.describe())
# param = {'feature_variables': ['domain'], 'target_variables': ['benign', 'dga']}
# print(param)

## Stage 2 - create and initialize a model

In [20]:
def init(df, param):

    mapping = {0: 'benign', 1: 'dga'}
    
    model = {
        "input_size": 67,
        "hidden_dim": 30,
        "embedding_dim": 5,
        "num_classes": 1,
        "n_layers": 2,
        "learning_rate": 0.001,
        "mapping": mapping,
        "num_epochs": int(param['options']['params']['epochs']),
        "batch_size": int(param['options']['params']['batch_size']),
    }
    
    model["train_dl"], model["test_dl"] = _get_train_test_data_loader(int(param['options']['params']['batch_size']), df)

    print("FIT build RNN model with input size " + str(model["train_dl"].dataset.__len__()))

    # Initialize DGA Classifier
    model['model'] = DGAClassifier(
        input_features=model["input_size"], 
        hidden_dim=model["hidden_dim"], 
        n_layers=model["n_layers"], 
        output_dim=model["num_classes"],
        embedding_dim=model["embedding_dim"], 
        batch_size=model["batch_size"]
    )
    
    # Define loss and optimizer
    model['criterion'] = torch.nn.BCELoss()
    
    model['optimizer'] = torch.optim.Adam(model['model'].parameters())
    
    return model

In [21]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df, param)
print(model)

Getting test and train data loaders.
FIT build RNN model with input size 16000
{'input_size': 67, 'hidden_dim': 30, 'embedding_dim': 5, 'num_classes': 1, 'n_layers': 2, 'learning_rate': 0.001, 'mapping': {0: 'benign', 1: 'dga'}, 'num_epochs': 2, 'batch_size': 32, 'train_dl': <torch.utils.data.dataloader.DataLoader object at 0x7f0a37867518>, 'test_dl': <torch.utils.data.dataloader.DataLoader object at 0x7f0a37867390>, 'model': DGAClassifier(
  (embedding): Embedding(67, 5)
  (rnn): RNN(5, 30, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (sigmoid): Sigmoid()
), 'criterion': BCELoss(), 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)}


## Stage 3 - fit the model

In [22]:
import time

# train your model
# returns a fit info json object and may modify the model object
def update_stats(accuracy, confusion_matrix, output, y):
    output = torch.round(output).flatten()
    equal = torch.eq(output, y)
    correct = int(torch.sum(equal))
    for j, i in zip(output, y):
        confusion_matrix[int(i),int(j)]+=1

    return accuracy + correct, confusion_matrix

def fit(model, param):
    
    returns = {"message": "model trained"}

    cuda = torch.cuda.is_available()
    device = torch.device("cpu") if not cuda else torch.device("cuda:0")
    seed_everything(seed=1337, cuda=cuda)
    
    accuracy, confusion_matrix = 0, np.zeros((2, 2), dtype=int)
    
    for epoch in range(1, model['num_epochs'] + 1):

        # Train
        model['model'].train()
        print("Train ({})".format(epoch))
        print("-"*20)
        t = time.time()

        accuracy, confusion_matrix = 0, np.zeros((2, 2), dtype=int)
        
        total_loss = 0
        
        # Iterate over dataset
        for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["train_dl"]):
            
            # Clear stored gradient
            model["optimizer"].zero_grad()

            # Forward pass
            output =  model['model'](x_padded, x_lens)
            loss = model['criterion'](output, torch.Tensor(y_padded).unsqueeze(1))

            total_loss += float(loss)
            
            accuracy, confusion_matrix = update_stats(
                accuracy, 
                confusion_matrix, 
                torch.Tensor(output), 
                torch.Tensor(y_padded)
            )
            
            # Backward pass
            loss.backward()

            # Update parameters
            model["optimizer"].step()
                            
            print("[Batch]: {}/{} in {:.5f} seconds".format(batch_num, len(model["train_dl"]), time.time() - t), end='\r', flush=True)
            t = time.time()
            
        print("")
        print("[Loss]: {:.5f}".format(total_loss / len(model["train_dl"])))
        print("[Accuracy]: {}/{} : {:.3f}%".format(
            accuracy, len(model["train_dl"].dataset), accuracy / len(model["train_dl"].dataset) * 100))
        print(confusion_matrix, "\n")
        
        # Evaluate
        model['model'].eval()
        accuracy, confusion_matrix = 0, np.zeros((2, 2), dtype=int)
        t = time.time()
        total_loss = 0
        print("Validation ({})".format(epoch))
        print("-"*20)
        with torch.no_grad():
            for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["test_dl"]):
                output =  model['model'](x_padded, x_lens)
                total_loss += float(model['criterion'](output, torch.Tensor(y_padded).unsqueeze(1)))
                accuracy, confusion_matrix = update_stats(
                    accuracy, 
                    confusion_matrix, 
                    torch.Tensor(output), 
                    torch.Tensor(y_padded)
                )
                print("[Batch]: {}/{} in {:.5f} seconds".format(batch_num, len(model["test_dl"]), time.time() - t), end='\r', flush=True)
                t = time.time()
                
        print("")
        print("[Loss]: {:.5f}".format(total_loss / len(model["test_dl"])))
        print("[Accuracy]: {}/{} : {:.3f}%".format(
            accuracy, len(model["test_dl"].dataset), accuracy / len(model["test_dl"].dataset) * 100))
        print(confusion_matrix, "\n")
        
    # memorize parameters
    returns['model_epochs'] = model['num_epochs']
    returns['model_batch_size'] = model['batch_size']
    returns['model_loss_acc'] = total_loss / len(model["train_dl"])
            
    return returns

In [23]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,param))

Train (1)
--------------------
[Batch]: 499/500 in 0.02798 seconds
[Loss]: 0.49018
[Accuracy]: 12046/16000 : 75.287%
[[4774 3286]
 [ 668 7272]] 

Validation (1)
--------------------
[Batch]: 124/125 in 0.00782 seconds
[Loss]: 0.35865
[Accuracy]: 3394/4000 : 84.850%
[[1389  551]
 [  55 2005]] 

Train (2)
--------------------
[Batch]: 499/500 in 0.03067 seconds
[Loss]: 0.03779
[Accuracy]: 15819/16000 : 98.869%
[[7918  142]
 [  39 7901]] 

Validation (2)
--------------------
[Batch]: 124/125 in 0.00752 seconds
[Loss]: 0.00784
[Accuracy]: 3994/4000 : 99.850%
[[1934    6]
 [   0 2060]] 

{'message': 'model trained', 'model_epochs': 2, 'model_batch_size': 32, 'model_loss_acc': 0.0019600566867738964}


## Stage 4 - apply the model

In [30]:
def apply(model, df, param):
    predict_dl = _get_predict_loader(int(param['options']['params']['batch_size']), df)    
    classes = model['mapping']
    model['model'].eval()    
    predictions = []
    
    with torch.no_grad():
        for batch_num, (x_padded,  x_lens) in enumerate(predict_dl):
            output =  model['model'](x_padded, x_lens)
            y_hat = torch.round(output.data)
            predictions += [classes[int(key)] for key in y_hat.flatten().numpy()]
           
    return predictions

In [33]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param)[:5])

Getting test and train data loaders.
['dga', 'dga', 'dga', 'dga', 'dga']


## Stage 5 - save the model

In [35]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    torch.save(model, MODEL_DIRECTORY + name + ".pt")
    return model

## Stage 6 - load the model

In [181]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = torch.load(MODEL_DIRECTORY + name + ".pt")
    return model

## Stage 7 - provide a summary of the model

In [41]:
# return model summary
def summary(model=None):
    returns = {"version": {"pytorch": torch.__version__} }
    if model is not None:
        if 'model' in model:
            returns["summary"] = str(model)
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code